# Sentiment Analysis: Word Embeddings and LSTM

In this notebook, we implement a sentiment analysis to associate stars (from 1 to 5) to the reviews of the [YelpReviewFull dataset](https://pytorch.org/text/stable/datasets.html#yelpreviewfull). 
This dataset consists of reviews from Yelp, and is extracted from the Yelp Dataset Challenge 2015 data. 
A data point of this dataset comprises a review's text and the corresponding label (1 to 5 stars).

For this, we will first use ..WORD EMBEDDINGS... TO...    [ResNet](https://arxiv.org/abs/1512.03385)

Then, we will use XXX... to...    [ResNet](https://arxiv.org/abs/1512.03385)

We achieve a XX% ...



In [46]:
%%bash
pip install torch==1.11.0
pip install folium==0.2.1
pip install torchdata
pip install datasets

  Using cached PyYAML-6.0-cp38-cp38-macosx_10_9_x86_64.whl (192 kB)


In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torchdata
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchtext import datasets, transforms, utils, models

## Data Preprocessing

To begin, we download the data with the [Hugging Face](https://huggingface.co/) library datasets.
(The direct loading from torchtext.datsets is impossible for now, given to a bug that will be corrected in the next version of PyTorch.)

In [62]:
from datasets import load_dataset
train_iter = load_dataset("yelp_review_full", split="train")
test_iter = load_dataset("yelp_review_full", split="test")

Reusing dataset yelp_review_full (/Users/louis/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
Reusing dataset yelp_review_full (/Users/louis/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


We check the number of reviews for the train and the test set.

In [66]:
print(f'Number of training reviews: {len(train_iter)}')
print(f'Number of testing reviews: {len(test_iter)}')

Number of training reviews: 650000
Number of testing reviews: 50000


In [63]:
def tokenize(label, line):
    return line.split()

tokens = []
for label, line in train_iter:
    tokens += tokenize(label, line)

In [70]:
print(train_iter[1])



{'label': 1, 'text': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars."}


In [44]:
import kaggle

kaggle.api.authenticate()

kaggle.api.dataset_download_files('The_name_of_the_dataset', path='the_path_you_want_to_download_the_files_to', unzip=True)


OSError: Could not find kaggle.json. Make sure it's located in /Users/louis/.kaggle. Or use the environment method.

In [41]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile


def download_and_unzip(url, extract_to='.'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)
    
url='https://github.com/huggingface/datasets/blob/ea1d9ec8badb707b7595f19643dddbe6e582e483/datasets/yelp_review_full/dummy/yelp_review_full/1.0.0/dummy_data.zip?raw=true'
download_and_unzip(url, extract_to='.')

In [39]:
import pandas as pd

url='https://github.com/huggingface/datasets/blob/ea1d9ec8badb707b7595f19643dddbe6e582e483/datasets/yelp_review_full/dummy/yelp_review_full/1.0.0/dummy_data.zip?raw=true'

df = pd.read_csv(url, compression='gzip',
                 header=0, sep=',', quotechar='"',
                 engine = 'python')

BadGzipFile: Not a gzipped file (b'PK')

In [27]:
import urllib.request
import tarfile
thetarfile = "ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/b0/ac/Breast_Cancer_Res_2001_Nov_9_3(1)_61-65.tar.gz"
ftpstream = urllib.request.urlopen(thetarfile)
thetarfile = tarfile.open(fileobj=ftpstream, mode="r|gz")




ReadError: not a gzip file

In [31]:
import requests

url = 'https://drive.google.com/uc?export=download&id=0Bz8a_Dbh9QhbZlU4dXhHTFhZQU0'
target_path = 'yelp_review_full_csv.tar.gz'

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        thetarfile = f.write(response.raw.read())


print(thetarfile)

1030


We next prepare the data for the analysis.

In [25]:
from string import punctuation



/Users/louis/opt/anaconda3/envs/profit/lib/python3.8/site-packages/torch/utils/data/datapipes/utils/common.py:24: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn(
/Users/louis/opt/anaconda3/envs/profit/lib/python3.8/site-packages/torch/utils/data/datapipes/iter/selecting.py:54: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn("Lambda function is not supported for pickle, please use "
